In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils
# from keras.optimizers import SGD

# import data

In [3]:
x_train=[]
y_train=[]
x_test=[]
y_test=[]
x_val=[]
y_val=[]

dim=128

path_train='../input/einterface-image-dataset/eINTERFACE_2021_Image/train'
path_test='../input/einterface-image-dataset/eINTERFACE_2021_Image/test'
path_val='../input/einterface-image-dataset/eINTERFACE_2021_Image/val'

category={'Anger':0,'Disgust':1,'Fear':2,'Happiness':3,'Sadness':4,'Surprise':5}


In [4]:

for i in sorted(os.listdir(path_train)):
#   print(i)
    path=os.path.join(path_train,i)
    for j in os.listdir(path):
        img=cv2.imread(os.path.join(path,j),0) #grayscaling
        img=cv2.resize(img,(dim,dim)) #resizing each image for uniformity
        x_train.append(img)
        y_train.append(category[i]) #adding labels for each image
        
print('Number of Training images:',len(x_train))
print('Number of Labels for training images:',len(y_train))


In [5]:

for i in sorted(os.listdir(path_test)):
#     print(i)
    path=os.path.join(path_test,i)
    for j in os.listdir(path):
        img=cv2.imread(os.path.join(path,j),0) #grayscaling
        img=cv2.resize(img,(dim,dim)) #resizing each image for uniformity
        x_test.append(img)
        y_test.append(category[i]) #adding labels for each image
        
print('Number of Testing images:',len(x_test))
print('Number of Labels for testing images:',len(y_test))


In [6]:
for i in sorted(os.listdir(path_val)):
    #print(i)
    path=os.path.join(path_val,i)
    for j in os.listdir(path):
        img=cv2.imread(os.path.join(path,j),0) #grayscaling
        img=cv2.resize(img,(dim,dim)) #resizing each image for uniformity
        x_val.append(img)
        y_val.append(category[i]) #adding labels for each image
        
print('Number of Validation images:',len(x_val))

print('Number of Labels for validation images:',len(y_val))

In [7]:
import cv2
import matplotlib.pyplot as plt

image = cv2.imread('../input/einterface-image-dataset/eINTERFACE_2021_Image/train/Anger/18_jpg.rf.b4b96f556d2cb2720c97c887bbaa3ec3.jpg')
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()

In [8]:
x_train=np.array(x_train)/255
x_test=np.array(x_test)/255
x_val=np.array(x_val)/255

In [9]:
y_train= np.array(y_train) 
y_test=np.array(y_test)
y_val=np.array(y_val)


In [10]:
y_train=np_utils.to_categorical(y_train)
y_test=np_utils.to_categorical(y_test)
y_val=np_utils.to_categorical(y_val)

print(y_train)

In [11]:
x_test.shape

In [12]:
x_train.shape

In [13]:
x_train_reshape=x_train.reshape(x_train.shape[0],dim,dim,1)
x_test_reshape=x_test.reshape(x_test.shape[0],dim,dim,1)
x_val_reshape=x_val.reshape(x_val.shape[0],dim,dim,1)

In [14]:
x_train_reshape.shape,y_train.shape,x_val_reshape.shape,y_val.shape

In [15]:
print(x_train_reshape.shape)
print(x_test_reshape.shape)
print(x_val_reshape.shape)

# model building

In [16]:
model=keras.Sequential()
model.add(keras.layers.Conv2D(32,(3,3),input_shape=(dim,dim,1),activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Conv2D(64,(3,3),activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Conv2D(64,(3,3),activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Conv2D(128,(3,3),activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Conv2D(256,(3,3),activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Conv2D(128,(3,3),activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Conv2D(128,(3,3),activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1024,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(6,activation='softmax'))

In [17]:
model.summary()

In [18]:
epochs=50
learning_rate=0.01
decay_rate=learning_rate/epochs
momentum=0.8
sgd=keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=False, name="SGD")

In [19]:
callback=keras.callbacks.EarlyStopping(monitor='val_loss',patience=3)


In [20]:
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [21]:
x_train_reshape.shape,y_train.shape,x_val_reshape.shape,y_val.shape

In [22]:
model_history=model.fit(x_train_reshape,y_train,batch_size=32,epochs=epochs,validation_data=(x_val_reshape,y_val),shuffle=True,callbacks=callback)

In [24]:
pd.DataFrame(model_history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

In [28]:
# Testing the model
loss,acc=model.evaluate(x_test_reshape,y_test)

In [31]:
pred=model.predict(x_test_reshape)

In [53]:
pred=np.argmax(pred,axis=1)

In [54]:
pred

In [41]:
x=[]
for i in pred:
    x.append(np.argmax(i))

In [48]:
x=np.array(x)

In [57]:
y_test=np.argmax(y_test,axis=1)

In [61]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,pred)

In [59]:
from sklearn.metrics import classification_report
print(classification_report(y_test, x))

In [62]:
import seaborn as sn
sn.heatmap(cm,annot=True,fmt='d')

In [83]:
from tensorflow.keras.models import save_model
save_model(model, "mymodel/final_model.h5")

In [81]:
from numpy import loadtxt
from tensorflow.keras.models import load_model
# load model
model1 = load_model('mymodel/final_model.h5')
# summarize model.
model1.summary()

In [82]:
model1.predict(x_test_reshape)